# Australian Open 2020: Winner Predictor
## Web Intelligence Course, Ca' Foscari University, A.Y. 2019/2020
#### Diego Chinellato, 867637 - Giorgia Campardo, 867928

In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from time import time

In [2]:
data_types = {
    'ATP': int,
    'Location': object,
    'Tournament': object,
    'Series': object,
    'Court': object,
    'Surface': object,
    'Round': object,
    'Best of': int,
    'Winner': object,
    'Loser': object,
    'WRank': float,
    'LRank': float,
    'WPts': float,
    'LPts': float,
    'W1': float,
    'L1': float,
    'W2': float,
    'L2': float,
    'W3': float,
    'L3': float,
    'W4': float,
    'L4': float,
    'W5': float,
    'L5': float,
    'Wsets': float,
    'Lsets': float,
    'Comment': object,
    'B365W': float,
    'B365L': float,
    'EXW': object,
    'EXL': float,
    'LBW': float,
    'LBL': float,
    'PSW': float,
    'PSL': float,
    'SJW': float,
    'SJL': float,
    'MaxW': float,
    'MaxL': float,
    'AvgW': float,
    'AvgL': float,
    'WElo': float,
    'WSurfElo': float,
    'WHand': object,
    'WBHand': float,
    'LElo': float,
    'LSurfElo': float,
    'LHand': object,
    'LBHand': float
}
full_dataset = pd.read_csv('data/dataset.csv', 
                            encoding='utf-8-sig', 
                            dtype=data_types,
                            parse_dates=['Date', 'WBD', 'LBD'])

In [3]:
full_dataset.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,WElo,WSurfElo,WBD,WHand,WBHand,LElo,LSurfElo,LBD,LHand,LBHand
0,1,Brisbane,Brisbane International,2011-01-02,ATP250,Outdoor,Hard,1st Round,3,Lopez F.,...,1783.4,1724.7,NaT,NaN,NaN,NaN,NaN,1984-03-24,R,1.0
1,1,Brisbane,Brisbane International,2011-01-02,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,...,1610.9,1611.0,1986-09-07,R,2.0,NaN,NaN,1974-05-05,L,2.0
2,2,Chennai,Chennai Open,2011-01-03,ATP250,Outdoor,Hard,1st Round,3,Kendrick R.,...,NaN,NaN,1979-11-15,R,2.0,NaN,NaN,1981-08-16,L,2.0
3,3,Doha,Qatar Exxon Mobil Open,2011-01-03,ATP250,Outdoor,Hard,1st Round,3,Bubka S.,...,NaN,NaN,1987-02-10,R,2.0,NaN,NaN,1985-08-07,R,2.0
4,1,Brisbane,Brisbane International,2011-01-03,ATP250,Outdoor,Hard,1st Round,3,Roddick A.,...,NaN,NaN,1982-08-30,R,2.0,NaN,NaN,1985-08-08,R,2.0


In [4]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 52 columns):
ATP           23634 non-null int32
Location      23634 non-null object
Tournament    23634 non-null object
Date          23634 non-null datetime64[ns]
Series        23634 non-null object
Court         23634 non-null object
Surface       23634 non-null object
Round         23634 non-null object
Best of       23634 non-null int32
Winner        23634 non-null object
Loser         23634 non-null object
WRank         23624 non-null float64
LRank         23586 non-null float64
WPts          23626 non-null float64
LPts          23587 non-null float64
W1            23483 non-null float64
L1            23485 non-null float64
W2            23260 non-null float64
L2            23260 non-null float64
W3            11173 non-null float64
L3            11173 non-null float64
W4            2200 non-null float64
L4            2200 non-null float64
W5            816 non-null float64
L5          

In [5]:
full_dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
ATP,23634.0,33.036896,18.040228,1.000,19.00,33.00,49.00,67.00
Best of,23634.0,3.386139,0.789431,3.000,3.00,3.00,3.00,5.00
WRank,23624.0,56.754614,70.940513,1.000,16.00,40.00,74.00,1890.00
LRank,23586.0,87.187908,109.198180,1.000,34.00,62.00,100.00,2159.00
WPts,23626.0,1988.829002,2477.960179,1.000,693.00,1060.00,2030.00,16950.00
LPts,23587.0,1127.641497,1294.122302,1.000,557.00,788.00,1195.00,16950.00
W1,23483.0,5.806583,1.225518,0.000,6.00,6.00,6.00,7.00
L1,23485.0,4.105770,1.834737,0.000,3.00,4.00,6.00,7.00
W2,23260.0,5.784007,1.246084,0.000,6.00,6.00,6.00,7.00
L2,23260.0,3.952193,1.857456,0.000,3.00,4.00,6.00,7.00


In [6]:
def compute_elo_rankings(data):
    """
    Given the list on matches in chronological order, for each match, computes 
    the elo ranking of the 2 players at the beginning of the match
    """
    print("Elo rankings computing...")
    players=list(pd.Series(list(data.Winner)+list(data.Loser)).value_counts().index)
    elo=pd.Series(np.ones(len(players))*1500,index=players)
    ranking_elo=[(1500,1500)]
    for i in range(1,len(data)):
        w=data.iloc[i-1,:].Winner
        l=data.iloc[i-1,:].Loser
        elow=elo[w]
        elol=elo[l]
        pwin=1 / (1 + 10 ** ((elol - elow) / 400))    
        K_win=32
        K_los=32
        new_elow=elow+K_win*(1-pwin)
        new_elol=elol-K_los*(1-pwin)
        elo[w]=new_elow
        elo[l]=new_elol
        ranking_elo.append((elo[data.iloc[i,:].Winner],elo[data.iloc[i,:].Loser])) 
    ranking_elo=pd.DataFrame(ranking_elo,columns=["elo_winner","elo_loser"])    
    ranking_elo["proba_elo"]=1 / (1 + 10 ** ((ranking_elo["elo_loser"] - ranking_elo["elo_winner"]) / 400))   
    return ranking_elo

In [7]:
def preprocess_data(min_date=2011,
                    max_date=2019,
                    features_to_drop=[], 
                    missing_values="drop", 
                    drop_first=False):
    """
    Processes raw data and returns a tuple (X, Y) where X is the cleaned dataset and Y is the array of labels.
    """
    # Loads data for the given years
    if max_date > 2019 or min_date < 2011:
        raise ValueError("Wrong date parameter")
    df = pd.read_csv("data/" + str(min_date) + ".csv", encoding='utf-8-sig', dtype=data_types)
    for year in range (min_date + 1, 2020):
        filename = "data/" + str(year) + ".csv"
        df = pd.concat((df, pd.read_csv(filename, encoding='utf-8-sig', dtype=data_types)))
    
    # Sort by date to calculate ELO
    X = df.sort_values(by='Date')
    
    # Drop unuseful columns
    features_to_drop += ['ATP', 'Location', 'Tournament', 'Date', 'Comment', 
                         'Winner', 'Loser', 'Wsets', 'Lsets', 
                         'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 
                         'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW', 'PSL', 'SJW', 'SJL',
                         'WBD', 'LBD']
    X = X.drop(columns=features_to_drop)
    
    # Deal with missing values
    X['WRank'] = X['WRank'].fillna(value=X['WRank'].max()+100).astype(int)
    X['LRank'] = X['LRank'].fillna(value=X['LRank'].max()+100).astype(int)

    if missing_values == 'drop':
        X = X.dropna()
    elif missing_values == 'custom':
        pass
    else:
        raise ValueError('Wrong parameter: missing_values')

    # Convert ordinal features to int (higher value means more important)
    series = ['ATP250', 'ATP500', 'Masters 1000', 'Masters Cup', 'Grand Slam']
    series2int = {s: i for i, s in enumerate(series)}
    rounds2int = {'1st Round': 0,
                  '2nd Round': 1,
                  '3rd Round': 2,
                  '4th Round': 3,
                  'Round Robin': 4,
                  'Quarterfinals': 5,
                  'Semifinals': 6,
                  'The Final': 7,
                 }
    X = X.replace({'Round': rounds2int, 'Series': series2int})
    
    # Convert categorical (binary) fields to int
    X = X.replace({'Court': {'Outdoor': 0, 'Indoor': 1}, 
                   'WHand': {'R': 0, 'L': 1}, 
                   'LHand': {'R': 0, 'L': 1}})
    X.astype({'WBHand': int, 'LBHand': int})
    
    # One hot encode categorical features into binary features
    X = pd.get_dummies(X, prefix=['Surface_'], columns=['Surface'], drop_first=drop_first)
    
    # Generate labels
    Y = np.concatenate([np.ones(X.shape[0], dtype=int), np.zeros(X.shape[0], dtype=int)])
    
    # Duplicate data with swapped columns
    tmp = X.copy()
    cols_to_swap = ['WRank', 'LRank', 'MaxW', 'MaxL',  'AvgW',  'AvgL', 'WPts', 'LPts',
                    'WElo', 'LElo', 'WSurfElo', 'LSurfElo', 'WHand', 'LHand', 'WBHand', 'LBHand']
    cols_to_swap = [f for f in cols_to_swap if f not in features_to_drop]
    cols_swapped = ['LRank', 'WRank', 'MaxL', 'MaxW',  'AvgL',  'AvgW', 'LPts', 'WPts',
                    'LElo', 'WElo', 'LSurfElo', 'WSurfElo', 'LHand', 'WHand', 'LBHand', 'WBHand']
    cols_swapped = [f for f in cols_swapped if f not in features_to_drop]
    
    tmp[cols_to_swap] = tmp[cols_swapped]
    tmp.index = np.array(range(X.shape[0] + 1, X.shape[0] * 2 + 1))
    X = pd.concat((X, tmp))
    
    # Generate new columns
    X['GreaterRank'] = (X['WRank'] < X['LRank']).astype(int)
    
    # Rename columns
    X = X.rename(columns={'WRank':'P1Rank', 'LRank':'P2Rank', 
                          'MaxW':'MaxP1', 'MaxL':'MaxP2', 
                          'AvgW':'AvgP1', 'AvgL':'AvgP2'})
    return X, Y

In [8]:
X, Y = preprocess_data(max_date=2011,
                       features_to_drop=[])

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16866 entries, 16 to 16866
Data columns (total 24 columns):
Series            16866 non-null int64
Court             16866 non-null int64
Round             16866 non-null int64
Best of           16866 non-null int32
P1Rank            16866 non-null int32
P2Rank            16866 non-null int32
WPts              16866 non-null float64
LPts              16866 non-null float64
MaxP1             16866 non-null float64
MaxP2             16866 non-null float64
AvgP1             16866 non-null float64
AvgP2             16866 non-null float64
WElo              16866 non-null float64
WSurfElo          16866 non-null float64
WHand             16866 non-null int64
WBHand            16866 non-null float64
LElo              16866 non-null float64
LSurfElo          16866 non-null float64
LHand             16866 non-null int64
LBHand            16866 non-null float64
Surface__Clay     16866 non-null uint8
Surface__Grass    16866 non-null uint8
Surface_

In [10]:
X.head()

,Series,Court,Round,Best of,P1Rank,P2Rank,WPts,LPts,MaxP1,MaxP2,...,WHand,WBHand,LElo,LSurfElo,LHand,LBHand,Surface__Clay,Surface__Grass,Surface__Hard,GreaterRank
16,0,0,0,3,98,14,554.0,2300.0,3.40,1.45,...,0,2.0,1892.6,1840.0,0,2.0,0,0,1,0
8,0,0,0,3,81,82,613.0,611.0,1.81,2.20,...,0,2.0,1597.1,1563.8,1,2.0,0,0,1,1
37,0,0,0,3,112,45,516.0,960.0,3.94,1.33,...,0,2.0,1850.6,1792.1,0,2.0,0,0,1,0
28,0,0,0,3,73,164,670.0,328.0,1.72,2.80,...,0,1.0,1614.3,1568.8,0,2.0,0,0,1,1
27,0,0,0,3,28,46,1430.0,945.0,1.45,3.30,...,0,2.0,1537.7,1481.7,0,1.0,0,0,1,1


In [11]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Series,16866.0,1.490454,1.503568,0.00,0.00,1.00,2.00,4.0
Court,16866.0,0.178584,0.383015,0.00,0.00,0.00,0.00,1.0
Round,16866.0,1.723230,2.174864,0.00,0.00,1.00,2.00,7.0
Best of,16866.0,3.408633,0.806426,3.00,3.00,3.00,3.00,5.0
P1Rank,16866.0,61.002016,76.583645,1.00,18.00,44.00,80.00,1890.0
P2Rank,16866.0,61.002016,76.583645,1.00,18.00,44.00,80.00,1890.0
WPts,16866.0,1919.629136,2502.473257,1.00,669.00,1009.00,1844.25,16950.0
LPts,16866.0,1919.629136,2502.473257,1.00,669.00,1009.00,1844.25,16950.0
MaxP1,16866.0,5.880784,324.466876,1.01,1.45,2.00,3.25,42136.0
MaxP2,16866.0,5.880784,324.466876,1.01,1.45,2.00,3.25,42136.0


Models construction

In [ ]:
def timeit(fun):
    # This is a decorator function used to log the model construction execution time
    def timed(*args, **kwargs):
        start = time()
        result = fun(*args, **kwargs)
        end = time()
        print('Execution took {:.2f} min'.format((end-start)/60))
        return result
    return timed

In [ ]:
def baseline_model(X, Y):
    # This model will always predict the winner as the player with the highest rank.
    # It's the lower bound on accuracy that we wish to improve
    y_pred = (X['P1Rank'] > X['P2Rank']).astype(int)
    accuracy = round((y_pred == Y).sum()/len(Y), 2)
    return accuracy
    
print('Accuracy for the baseline model is:', baseline_model(X, Y))

In [ ]:
@timeit
def build_decision_tree(X_train, Y_train, X_valid, Y_valid, draw_graph=False):
    from sklearn.tree import DecisionTreeClassifier, export_graphviz
    # Builds a decision tree and performs hyper-parameters tuning
    scores = []
    criterions = ('gini', 'entropy')
    splitters = ("best", "random")
    depths = list(range(3, 100, 10)) + [None]
    leaves = list(range(10, 201, 30)) + [None]
    for criterion in criterions:
        for splitter in splitters:
            for depth in depths:
                for max_leaves in leaves:
                    dt = DecisionTreeClassifier(max_leaf_nodes=max_leaves,
                                                splitter=splitter,
                                                criterion=criterion,
                                                max_depth=depth)
                    dt.fit(X_train, Y_train)
                    valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=dt.predict(X_valid)), 3)
                    scores += [(valid_acc, criterion, depth, leaves)]
    best = max(scores)
    acc, criterion, depth, leaves = best
    print('Max accuracy on validation set:', acc)
    print('Criterion:', criterion)
    print('Max depth:', depth)
    print('Max leaves:', leaves)
    dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                 criterion=criterion,
                                 max_depth=depth)
    dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    if draw_graph:
        dot_data = export_graphviz(dt, out_file=None, 
                                    feature_names=X_train.columns, class_names=True,  
                                    filled=True, rounded=True, special_characters=True)  
        graph = graphviz.Source(dot_data)
        display(graph)
    return dt, best

In [ ]:
@timeit
def build_bagging_classifier(X_train, Y_train, X_valid, Y_valid):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    scores = []
    for bootstrap in (True, False):
        for n_est in range(10, 201, 20):
            for max_samples in (0.25, 0.50, 0.75, 1.0):
                for criterion in ('gini', 'entropy'):
                    dt = DecisionTreeClassifier(criterion=criterion)
                    bagged_dt = BaggingClassifier(dt, bootstrap=bootstrap,
                                                  n_estimators=n_est,
                                                  max_samples=max_samples,
                                                  n_jobs=-1)
                    bagged_dt.fit(X_train, Y_train)
                    valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=bagged_dt.predict(X_valid)), 3)
                    scores += [(valid_acc, bootstrap, n_est, max_samples, criterion)]
    best = max(scores)
    acc, bootsrap, n_est, max_samples, criterion = best
    print('Max accuracy on validation set:', acc)
    print('Boostrap:', bootsrap)
    print('N. estimators:', n_est)
    print('Max samples:', max_samples)
    print('Tree criterion:', criterion)
    bagged_dt = BaggingClassifier(dt, 
                                  bootstrap=bootstrap,
                                  n_estimators=n_est, 
                                  max_samples=max_samples)
    bagged_dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return bagged_dt, best

In [ ]:
@timeit
def build_adaboost(X_train, Y_train, X_valid, Y_valid):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import AdaBoostClassifier
    scores = []
    for n_est in range(50, 301, 50):
        for learning_rate in (0.50, 0.75, 1.0, 1.5):
            for criterion in ('gini', 'entropy'):
                for depth in range(5, 21, 10):
                    for leaves in range(5, 100, 25):
                        dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                                    criterion=criterion,
                                                    max_depth=depth)
                        boosted_dt = AdaBoostClassifier(dt,
                                                        n_estimators=n_est,
                                                        learning_rate=learning_rate)
                        boosted_dt.fit(X_train, Y_train)
                        valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=bagged_dt.predict(X_valid)), 3)
                        scores += [(valid_acc, n_est, learning_rate, leaves, criterion, depth)]
    best = max(scores)
    acc, n_est, learning_rate, leaves, criterion, depth = best
    print('Max accuracy on validation set:', acc)
    print('N. estimators:', n_est)
    print('Learning rate:', learning_rate)
    print('Tree max leaves:', leaves)
    print('Tree max depth:', depth)
    print('Tree criterion:', criterion)
    dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                criterion=criterion,
                                max_depth=depth)
    boosted_dt = AdaBoostClassifier(dt,
                                    n_estimators=n_est,
                                    learning_rate=learning_rate)
    boosted_dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return boosted_dt, best

In [ ]:
@timeit
def build_random_forest(X_train, Y_train, X_valid, Y_valid):
    from sklearn.ensemble import RandomForestClassifier
    scores = []
    for n_est in range(50, 501, 50):
        for criterion in ('gini', 'entropy'):
            for bootstrap in (True, False):
                for n_features in (None, 'sqrt', 'log2'):
                    rf = RandomForestClassifier(n_estimators=n_est,
                                                bootstrap=bootstrap,
                                                criterion=criterion,
                                                max_features=n_features,
                                                n_jobs=-1)
                    rf.fit(X_train, Y_train)
                    valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=rf.predict(X_valid)), 3)
                    scores += [(valid_acc, n_est, criterion, bootstrap, n_features)]
    best = max(scores)
    acc, n_est, criterion, bootstrap, features = best
    print('Max accuracy on validation set:', acc)
    print('N. estimators:', n_est)
    print('Criterion:', criterion)
    print('Bootstrap:', bootstrap)
    print('Features criterion (None means all features):', features)
    rf = RandomForestClassifier(n_estimators=n_est,
                               bootstrap=bootstrap,
                               criterion=criterion,
                               n_jobs=-1)
    rf.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return rf, best

Let's split the data into train set, validation set and test set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.25)

In [ ]:
dt, dt_params = build_decision_tree(X_train, Y_train, X_valid, Y_valid)

In [ ]:
bagged_dt, bagged_params = build_bagging_classifier(X_train, Y_train, X_valid, Y_valid)

In [ ]:
boosted_dt, boosted_params = build_adaboost(X_train, Y_train, X_valid, Y_valid)

In [ ]:
rf, rf_params = build_random_forest(X_train, Y_train, X_valid, Y_valid)

In [ ]:
def feature_importance_rf(X, rf):
    fig, ax = plt.subplots(figsize=(6,6))
    ax.bar(range(0,X.shape[1]), rf.feature_importances_)
    ax.set_title("Feature Importances")
    for i, f in enumerate(rf.feature_importances_):
        print('{:2}'.format(i), ' -> ', X.columns[i])

In [12]:
def model_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    zz = [ [xx,yy] for xx in np.linspace(x_min, x_max, 40) 
                   for yy in np.linspace(y_min, y_max, 40) ]
    zz = np.array(zz)
    z_labels = model.predict(zz)

    plt.figure()
    plt.scatter(zz[:,0], zz[:,1], c=z_labels, marker='+', alpha=0.3)
    plt.scatter(X[:,0], X[:,1], c=y, alpha=0.6)

NameError: name 'model' is not defined

In [ ]:
feature_importance_rf(X_train, rf)

In [ ]:
def report(X, Y, models):
    for model in models:
        print('Algorithm:', str(type(model)).split('.')[-1][:-2])
        rep = classification_report(y_true=Y, y_pred=model.predict(X))
        print(rep)
        print()

In [ ]:
report(X_test, Y_test, [dt, bagged_dt, boosted_dt, rf])